<h1 style="color:blue;">Video Game Industry Analysis 2000–2013: Platforms, Genres, and Ratings</h1>

- Author: Maria Gryaznova
- Date: 19.12.2024

### Project Goals and Objectives

#### Project Goal

Prepare and analyze video game sales data to identify key patterns and determine the platforms, genres, and other factors influencing game popularity and the development of the gaming industry from 2000 to 2013.

#### Objectives

1. Data Loading and Exploration

    - Study the structure and volume of the data.
    - Assess missing values, data errors, and correctness of data types.

2. Data Preprocessing

    - Standardize the data format.
    - Correct data types.
    - Handle missing values and duplicates to improve data quality.

3. Data Filtering

    - Limit the analysis to games released between 2000 and 2013.

4. Data Categorization

    - Classify games based on user and critic ratings.
    - Identify the top 7 platforms with the highest number of released games.

5. Final Summary

### Data Description

The project will use data from the dataset `/datasets/new_games.csv` with the following description:
- `Name` — name of the game.
- `Platform` — name of the platform.
- `Year of Release` — year the game was released.
- `Genre` — genre of the game.
- `NA sales` — sales in North America (in millions of copies sold).
- `EU sales` — sales in Europe (in millions of copies sold).
- `JP sales` — sales in Japan (in millions of copies sold).
- `Other sales` — sales in other countries (in millions of copies sold).
- `Critic Score` — critic rating (from 0 to 100).
- `User Score` — user rating (from 0 to 10).
- `Rating` — ESRB rating (Entertainment Software Rating Board). This organization determines video game ratings and assigns appropriate age categories.

### Project Structure

1. Data Loading and Exploration
2. Error Checking and Data Preprocessing <br>
    2.1. Column Names or Labels in the DataFrame <br>
    2.2. Data Types <br>
    2.3. Presence of Missing Values <br>
    2.4. Explicit and Implicit Duplicates in the Data
3. Data Filtering
4. Data Categorization
5. Final Output

## 1. Loading data and exploring


In [55]:
# Importing pandas and numpy libraries
import pandas as pd
import numpy as np

In [56]:
# Loading data from the new_games.csv dataset into the games dataframe
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [57]:
# Output information about the dataframe
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [58]:
# Display the first lines of the dataframe on the screen
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


The dataset `new_games.csv` contains 11 columns and 16956 rows, representing information about video games.

Let's examine the data types and their correctness:
- **String Data.** Seven columns have the `object` data type:
    - The `Name` column contains string data (game names and platform names). The `object` data type is appropriate here.
    - The `EU sales` and `JP sales` columns have the `object` data type, but they should be represented as numeric values of type `float64`, since they represent sales in millions of copies (the data may contain incorrect values).
    - The `User Score` column also has the `object` type, but it should be represented as a numeric value of type `float64` (incorrect values, such as 'unknown', may exist).
    - For the columns `Genre`, `Rating`, and `Platform`, the `category` data type should be used since they contain a limited number of unique values.

- **Floating-Point Numbers.** Four columns have the `float64` data type:
    - `Year of Release` represents the years of game releases. It would be more appropriate to store this information in an integer format. The `int16` type is optimal for storage, as the range of values is fully covered.
    - The data type for the `NA sales`, `Other sales`, and `Critic Score` columns is correct and does not require changes.

Additionally, it is clear that most columns contain missing values, which need to be addressed before converting columns to the correct data types.

Also, not all column names are in a convenient format for working with them, so they should be converted to a consistent style.

---

## 2. Data Quality Check and Preprocessing

### 2.1. Column Names or Labels of the DataFrame

- At this stage, we check the writing style of the column names in the dataframe.

In [59]:
# Display current column names
print(games.columns)

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')


It is evident that the column names do not follow the `snake_case` style.

We will perform the necessary transformations: applying the `.str.lower()` method to convert the text to lowercase and `.str.replace(' ', '_')` to replace spaces with underscores.

In [60]:
# Convert column names to snake_case
games.columns = games.columns.str.lower().str.replace(' ', '_')

In [61]:
# Output new column names
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

As a result of this step, the dataframe now has correct and uniform column names in the `snake_case` style, which will make further work with the data easier.

### 2.2. Data Types

We have columns with clearly incorrect data types:
- `eu_sales` and `jp_sales` — currently `object`, they should be `float64`.
- `user_score` — currently `object`, it should be `float64`.

We will start working with these columns to convert them to numeric types and handle any incorrect values. Let's proceed step by step.

Let's begin with `eu_sales`:

In [62]:
# Check for unique values in the 'eu_sales' column
games['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

We discovered an incorrect value in the array — the string 'unknown'. To convert the `eu_sales` column to the `float64` type, we will use `.to_numeric`, and 'unknown' will be automatically replaced with NaN.

In [63]:
# Convert the column to a number format
games['eu_sales'] = pd.to_numeric(games['eu_sales'], errors='coerce')

We will do the same for the `jp_sales` column. If it contains any incorrect values, such as 'unknown', they will be automatically replaced with NaN when converting the column to `float64` using `.to_numeric`.

In [64]:
# Check for unique values in the 'jp_sales' column
games['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

We will convert the data format using `.to_numeric`, where 'unknown' will automatically be replaced with NaN.

In [65]:
# Convert the column to a numeric format
games['jp_sales'] = pd.to_numeric(games['jp_sales'], errors='coerce')

Working with the `user_score` column

In [66]:
games['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In the array, an incorrect value was found — the string 'tbd'.

In [67]:
# Convert the column to a numeric format
games['user_score'] = pd.to_numeric(games['user_score'], errors='coerce')

We need to convert the data in the `platform`, `genre`, and `rating` columns to categorical data type to improve performance and memory management:

In [68]:
# Convert the platform, genre and rating column variables to the 'category' type
games['platform'] = games['platform'].astype('category')
games['genre'] = games['genre'].astype('category')
games['rating'] = games['rating'].astype('category')

In [69]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16956 non-null  category
 2   year_of_release  16681 non-null  float64 
 3   genre            16954 non-null  category
 4   na_sales         16956 non-null  float64 
 5   eu_sales         16950 non-null  float64 
 6   jp_sales         16952 non-null  float64 
 7   other_sales      16956 non-null  float64 
 8   critic_score     8242 non-null   float64 
 9   user_score       7688 non-null   float64 
 10  rating           10085 non-null  category
dtypes: category(3), float64(7), object(1)
memory usage: 1.1+ MB


We can see that we have successfully converted the data in `eu_sales`, `jp_sales`, and `user_score` to the numeric `float64` format. Meanwhile, the `platform`, `genre`, and `rating` columns have been converted to categorical data types.

What remains is to change the data type of the `year_of_release` column, which currently has the `float64` type due to the presence of NaN values. However, for the release year of the game, it makes sense to store the data in the integer format `int16`. This conversion will be done during the handling of missing values, as attempting to directly convert it to an integer type at this stage will cause an error (integer values cannot contain NaN).

### 2.3. Missing Data

During the initial exploration of the data, it was found that some columns in the `new_games.csv` dataset contain missing values. Additionally, after transforming some data, the number of missing values may have increased.

Let’s take a closer look at the missing values.

In [70]:
# Count the gaps in each column
games.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [71]:
# Calculate the percentage of gap
games.isna().sum() / len(games) * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

- The columns `na_sales`, `other_sales` (sales in different regions), as well as the `platform` column, do not contain missing values.
- However, `eu_sales` and `jp_sales` contain a few missing values (less than 1% in both cases). We will replace them with the mean for each platform and year.
- The columns `name` and `genre` each have only 2 missing values (0.01%), which is not critical. These rows can be removed without significantly affecting further analysis.
- The column `year_of_release` contains 275 missing values (1.62%). Missing values may have occurred if data was collected from different sources, where the release year was not provided for some games. By removing these missing values, we will exclude games with unknown release years, ensuring data accuracy and facilitating further work.
- The columns `critic_score`, `user_score`, and `rating` have the largest number of missing values. Missing values in `rating` (40.52%) can be filled with the value "Unknown" or a similar placeholder, as we will convert this column to categorical data. Missing values in `critic_score` (51.39%) and `user_score` (40.13%) can be filled with the unrealistic placeholder `-1`.

Let's start by removing the missing values in `name`, `genre`, and `year_of_release`. We will use the `dropna` method with the `inplace=True` argument to make changes directly to the dataset.

In [72]:
# Remove rows with gaps in the columns 'name', 'genre' and 'year_of_release' 
games.dropna(subset=['name', 'genre', 'year_of_release'], inplace=True)

After removing the missing values in `year_of_release`, we will convert the data in this column to the `int16` type.

In [73]:
games['year_of_release'] = games['year_of_release'].astype('int16')

Now, we will fill the missing values in the `rating` column with the value "Unknown".

In [74]:
# Update categories in the 'rating' column to add 'Unknown'
games['rating'] = games['rating'].cat.add_categories('Unknown')

# Replace gaps with 'Unknown'
games['rating'].fillna('Unknown', inplace=True)

C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\1686138303.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games['rating'].fillna('Unknown', inplace=True)


Now, let's work with the missing values in the `critic_score` and `user_score` columns. We will replace them with `-1`.

In [75]:
# Replace blanks with -1 in 'critic_score' and 'user_score'
games['critic_score'].fillna(-1, inplace=True)
games['user_score'].fillna(-1, inplace=True)


C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\3586277314.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games['critic_score'].fillna(-1, inplace=True)
C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\3586277314.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

Finally, we need to fill the missing values in `eu_sales` and `jp_sales`, replacing them with the mean value grouped by platform and year.

In [76]:
# We calculate the average for 'eu_sales' by platform and year of release
mean_eu_sales = games.groupby(['platform', 'year_of_release'])['eu_sales'].transform('mean')

# We replace gaps with average
games['eu_sales'] = games['eu_sales'].fillna(mean_eu_sales)

# We calculate the average for 'jp_sales' by platform and year of release
mean_jp_sales = games.groupby(['platform', 'year_of_release'])['jp_sales'].transform('mean')

# We replace gaps with average
games['jp_sales'] = games['jp_sales'].fillna(mean_jp_sales)

C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\2197708599.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_eu_sales = games.groupby(['platform', 'year_of_release'])['eu_sales'].transform('mean')
C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\2197708599.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_jp_sales = games.groupby(['platform', 'year_of_release'])['jp_sales'].transform('mean')


In [77]:
# Displaying information about the dataframe
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16679 non-null  object  
 1   platform         16679 non-null  category
 2   year_of_release  16679 non-null  int16   
 3   genre            16679 non-null  category
 4   na_sales         16679 non-null  float64 
 5   eu_sales         16679 non-null  float64 
 6   jp_sales         16679 non-null  float64 
 7   other_sales      16679 non-null  float64 
 8   critic_score     16679 non-null  float64 
 9   user_score       16679 non-null  float64 
 10  rating           16679 non-null  category
dtypes: category(3), float64(6), int16(1), object(1)
memory usage: 1.1+ MB


In [78]:
games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,-1.0,-1.0,Unknown
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,-1.0,-1.0,Unknown


We can see that after all the transformations, the dataframe no longer contains missing values, and the number of rows decreased from 16,956 to 16,679.  
- Rows with missing values in the `name`, `genre`, `eu_sales`, `year_of_release`, and `jp_sales` columns were deleted.  
- In the `rating` column, missing values were replaced with "Unknown".  
- For `critic_score` and `user_score`, missing values were filled with the mean value by platform and year, and where that wasn't possible, a placeholder value of -1 was used.  
- The data type of `year_of_release` was converted to `int16`.

### 2.4. Explicit and implicit duplicates in data


Let's examine the unique values in the textual columns: `name`, `platform`, `genre`, and `rating`. We will check for potential subtle duplicates caused by typos or inconsistent naming conventions.

In [79]:
# Check the unique values in textual columns before normalization
print("Unique values before normalization:")
for col in ['name', 'platform', 'genre', 'rating']:
    if col in games.columns:
        print(f"{col}: {games[col].nunique()} Unique values")

Unique values before normalization:
name: 11426 Unique values
platform: 31 Unique values
genre: 24 Unique values
rating: 9 Unique values


For the `platform`, `genre`, and `rating` columns, we expected to see a small number of unique values, as they are categorical data. In the `name` column, we see 11,426 unique values out of 16,679.

Let's continue working with implicit duplicates.
- Convert `name` and `genre` to lowercase using `.str.lower()`.
- Convert `platform` and `rating` to uppercase using `.str.upper()`.
- Remove leading and trailing spaces from all strings using `.str.strip()`.

In [80]:
# Convert text data to lower or upper case, remove extra spaces
games['name'] = games['name'].str.lower().str.strip()
games['platform'] = games['platform'].str.upper().str.strip()
games['genre'] = games['genre'].str.lower().str.strip()
games['rating'] = games['rating'].str.upper().str.strip()


In [81]:
games['rating'].unique()

array(['E', 'UNKNOWN', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

It can be seen that the rating contains the value 'K-A', which is the original version of the 'E' rating. Let's replace the 'K-A' values with 'E'.

In [82]:
# Replace all 'K-A' values with 'E' in the 'rating' column
games['rating'] = games['rating'].replace('K-A', 'E')

In [83]:
# Output of the number of unique values after normalization
print("\nUnique values after normalization:")
for col in ['name', 'platform', 'genre', 'rating']:
    if col in games.columns:
        print(f"{col}: {games[col].nunique()} unique values")


Unique values after normalization:
name: 11426 unique values
platform: 31 unique values
genre: 12 unique values
rating: 8 unique values


We can see that after normalization, the number of unique values in `genre` was reduced by half, now it has 12 instead of 24, and in `rating`, it decreased by 1 value, now there are 8 unique ratings.
The number of unique values in other columns remained the same, as they originally didn't contain any implicit duplicates.

Let's continue working with duplicates and move on to finding and removing explicit duplicates in the data.

Let's check for exact duplicates — rows where all values are identical.  
If duplicates are found, we will remove them, as they are likely errors (e.g., accidental repetition of records during data collection).

In [84]:
# Check for complete duplicates
games[games.duplicated()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
268,batman: arkham asylum,PS3,2009,action,2.24,1.31,0.07,0.61,91.0,8.9,T
368,james bond 007: agent under fire,PS2,2001,shooter,1.90,1.13,0.10,0.41,72.0,7.9,T
717,god of war: ascension,PS3,2013,action,1.23,0.63,0.04,0.35,80.0,7.5,M
823,wipeout: the game,WII,2009,misc,1.94,0.00,0.00,0.12,-1.0,-1.0,UNKNOWN
848,rayman raving rabbids: tv party,WII,2008,misc,0.72,1.08,0.00,0.23,73.0,7.7,E10+
...,...,...,...,...,...,...,...,...,...,...,...
16671,fullmetal alchemist: prince of the dawn,WII,2009,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,UNKNOWN
16753,routes pe,PS2,2007,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,UNKNOWN
16799,transformers: prime,WII,2012,action,0.00,0.01,0.00,0.00,-1.0,-1.0,UNKNOWN
16912,metal gear solid v: the definitive experience,XONE,2016,action,0.01,0.00,0.00,0.00,-1.0,-1.0,M


In [85]:
#Removing obvious duplicates
games.drop_duplicates(inplace=True)

In addition to complete duplicates, we will also check for duplicates by name, platform and year of release.

In [86]:
# Find duplicates by name, platform and year of release
games[games.duplicated(subset=['name', 'platform', 'year_of_release'], keep=False)]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
606,madden nfl 13,PS3,2012,sports,2.11,0.22,0.0,0.23,83.0,5.5,E
16465,madden nfl 13,PS3,2012,sports,0.00,0.01,0.0,0.00,83.0,5.5,E


For the game 'Madden NFL 13', we see that most of the values match. We will keep the first row, as the data in it appears more complete and reliable, and remove the second one.

In [87]:
# We remove duplicates by checking by name, platform and year of release
games.drop_duplicates(subset=['name', 'platform', 'year_of_release'], keep='first', inplace=True)

# Checking the Size of a DataFrame After Removing Duplicates
games.shape

(16443, 11)

- During the data preparation process, we removed and transformed missing values and errors in the data, and also worked on transforming and removing duplicates. Let's calculate the number of rows removed in absolute and relative values.

In [88]:
# Number of lines before clearing
initial_rows = 16956

# Number of lines after clearing
final_rows = games.shape[0]

# Calculate the number of deleted rows
removed_rows = initial_rows - final_rows

# Calculate the percentage of deleted rows
removed_percentage = (removed_rows / initial_rows) * 100

removed_rows, removed_percentage


(513, 3.0254777070063694)

In the original dataset, there were 16,956 records and 11 columns.

Some columns contained missing values and had incorrect data types.

- All column names were converted to the `snake_case` style.
- Rows with missing values in the `name`, `genre`, and `year_of_release` columns were removed, as the percentage of missing values in these columns was small. Missing values in the `rating` column were replaced with 'Unknown'. Missing values in the `critic_score` and `user_score` columns were replaced with the unrealistic value `-1`. Missing values in `eu_sales` and `jp_sales` were replaced with the mean value grouped by platform and release year.
- The `year_of_release` column was converted to the `int16` data type for memory optimization and easier representation of the release year. The `eu_sales` and `jp_sales` columns were converted to numeric types, not strings, for correct data processing. The `platform`, `genre`, and `rating` columns were converted to the `category` data type.
- Transformations were applied to the `name`, `genre`, `platform`, and `rating` columns to eliminate possible implicit duplicates (such as different case or typos).
- Explicit duplicate rows were removed, resulting in a smaller dataset.

As a result of these operations, the dataset size decreased from 16,956 to 16,443 records, which corresponds to the removal of 513 rows (3%).
All columns now contain data without missing values, and data types have been optimized.

---

## 3. Data Filtering

Our colleagues are interested in studying the history of game sales in the early 21st century, specifically the period from 2000 to 2013. To do this, we will filter the data based on the release year and save the filtered subset into a new dataframe, `df_actual`.

In [89]:
# We apply a filter that limits the range of games output from 2000 to 2013 inclusive
df_actual = games[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)]

In [90]:
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12780 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12780 non-null  object 
 1   platform         12780 non-null  object 
 2   year_of_release  12780 non-null  int16  
 3   genre            12780 non-null  object 
 4   na_sales         12780 non-null  float64
 5   eu_sales         12780 non-null  float64
 6   jp_sales         12780 non-null  float64
 7   other_sales      12780 non-null  float64
 8   critic_score     12780 non-null  float64
 9   user_score       12780 non-null  float64
 10  rating           12780 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.1+ MB


In [91]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,WII,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
2,mario kart wii,WII,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,WII,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,wii play,WII,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E


12,771 games were released between 2000 and 2013.

---

## 4. Data Categorization

We need to categorize the data:

First, we will categorize all games based on user ratings (`user_score`) into the following categories:
- High rating (from 8 to 10 inclusive),
- Average rating (from 3 to 8, excluding the upper boundary),
- Low rating (from 0 to 3, excluding the upper boundary).

To do this, we will create a new column called `user_score_category`.

In [92]:
# Defining categories using a function
def categorize_user_score(score):
    if 8 <= score <= 10:
        return 'high rating'
    elif 3 <= score < 8:
        return 'average rating'
    elif 0 <= score < 3:
        return 'low rating'

df_actual['user_score_category'] = df_actual['user_score'].apply(categorize_user_score)

# Assign the column the 'category' data type
df_actual['user_score_category'] = df_actual['user_score_category'].astype('category')

C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\2252965003.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['user_score_category'] = df_actual['user_score'].apply(categorize_user_score)
C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\2252965003.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['user_score_category'] = df_actual['user_score_category'].astype('category')


Now, we will categorize all games based on critic ratings (`critic_score`) into the following categories:
- High rating (from 80 to 100 inclusive),
- Average rating (from 30 to 80, excluding the upper boundary),
- Low rating (from 0 to 30, excluding the upper boundary).

To do this, we will create a new column called `critic_score_category`.

In [93]:
def categorize_critic_score(score):
    if 80 <= score <= 100:
        return 'high rating'
    elif 30 <= score < 80:
        return 'average rating'
    elif 0 <= score < 30:
        return 'low rating'

df_actual['critic_score_category'] = df_actual['critic_score'].apply(categorize_critic_score)


# Assign the column the 'category' data type
df_actual['critic_score_category'] = df_actual['critic_score_category'].astype('category')

C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\885852705.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['critic_score_category'] = df_actual['critic_score'].apply(categorize_critic_score)
C:\Users\Maria\AppData\Local\Temp\ipykernel_15824\885852705.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['critic_score_category'] = df_actual['critic_score_category'].astype('category')


After categorizing the data, we will check the result by grouping the data by the newly created categories and counting the number of games in each category.

In [94]:
# Grouping by user rating categories
print("Number of games by user rating categories:")
user_score_counts = df_actual['user_score_category'].value_counts()
for category, count in user_score_counts.items():
    print(f"{category}: {count} games")

# Grouped by critics' ratings categories
print("\nNumber of games by critic ratings category:")
critic_score_counts = df_actual['critic_score_category'].value_counts()
for category, count in critic_score_counts.items():
    print(f"{category}: {count} games")

Number of games by user rating categories:
average rating: 4080 games
high rating: 2286 games
low rating: 116 games

Number of games by critic ratings category:
average rating: 5422 games
high rating: 1691 games
low rating: 55 games


- We will highlight the top-7 platforms based on the number of games released during the entire relevant period.

In [95]:
# We group by platforms and count the number of games
top_platforms = df_actual['platform'].value_counts().head(7)

print("Top 7 platforms by number of games:")
print(top_platforms)

Top 7 platforms by number of games:
platform
PS2     2127
DS      2120
WII     1275
PSP     1180
X360    1121
PS3     1086
GBA      811
Name: count, dtype: int64


Thus, the PS2 platform had the highest number of games released during the period from 2000 to 2013.

---

## 5. Final conclusion


The dataset `new_games.csv` contains **11 columns and 16956 rows**, presenting information about the sales of video games across various genres and platforms, as well as user and critic ratings. During the data inspection and preprocessing phase, the following steps were taken:

- Column names were standardized to the snake_case style.
- Rows with missing values in the `name`, `genre`, and `year_of_release` columns were removed. Missing values in the `rating` column were replaced with 'Unknown', and missing values in the `critic_score` and `user_score` columns were replaced with -1. Missing values in the `eu_sales` and `jp_sales` columns were replaced with the average value for each platform and year of release.
- Data types in several columns were converted: `year_of_release` was converted to `int16`, `eu_sales` and `jp_sales` were converted to numeric types, and `platform`, `genre`, and `rating` were converted to the `category` data type.
- Data transformations were applied to the `name`, `genre`, `platform`, and `rating` columns to eliminate potential implicit duplicates (e.g., due to different cases or typos). Explicit duplicates were removed, resulting in a reduced dataset size with corrected data.

As a result of the preprocessing, the dataset size was reduced from 16956 to **16443 rows**, corresponding to the removal of 513 rows (3%).

In the analysis of video game sales, a slice of data for the period **2000-2013** was selected. New columns `user_score_category` and `critic_score_category` were created to categorize games by user and critic ratings into three categories: high, medium, and low ratings. The top-7 platforms by the number of games released during this period were identified.

Key findings from the analysis:

- The highest number of games (4080) received a medium rating from **users**.
- A significant portion, 2286 games, received a high rating, indicating the presence of quality games.
- Only 116 games were categorized with a low user rating, which may indicate limited audience interest.

Regarding **critic ratings**:
- About 5422 games received an average rating from critics.
- 1691 games received a high rating, showing a relatively small percentage of highly rated titles.
- Only 55 games were classified as low-rated by critics, indicating a small number of unsuccessful games.

As for the **platforms**:
- PS2 and DS were the leaders, with 2127 and 2120 games released, respectively.
- Other platforms such as WII, PSP, X360, PS3, and GBA also developed strong game libraries during the period but had fewer games compared to the leading platforms.